# **HYPERPARAMETER TUNING - GOOGLE COLAB CODE**

**Bangkit Academy 2024 Batch 2 - MSIB 7**

**Team ID : C242-PS012**

**Team Member [Machine Learning] :**

**1. M296B4KY3091  –  Muhammad Sulthon  Haqiqi  –  Universitas  Pembangunan Nasional Veteran Jawa Timur**

**2. M007B4KY3020    –    Muhammad   Rakha    Almasah    –    Universitas    Dian Nuswantoro**

**3. M123B4KY0062   –   Achmad   Zahir   Wajdi   –  Politeknik   Elektronika   Negeri Surabaya**

# Installing Keras Tuner

In [ ]:
!pip install -q -U keras-tuner

# Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tempfile
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

# Load and Display Combined Data

In [ ]:
data_gabungan = pd.read_csv('/content/data_gabungan.csv')

In [ ]:
data_gabungan.head()

# Define Features (X) and Targets (Y)

In [ ]:
X = data_gabungan[['Close USDIDR', 'BI Rate', 'Inflasi']].values

Y = data_gabungan[['Close CTRA', 'Close INDF', 'Close ASII', 'Close BSDE',
                 'Close ICBP', 'Close KLBF', 'Close ITMG', 'Close JPFA',
                 'Close TLKM', 'Close ULTJ', 'Close ACES', 'Close TSPC',
                  'Close SMAR', 'Close SMSM', 'Close JRPT', 'Close DUTI',
                   'Close EPMT', 'Close SMCB', 'Close PWON', 'Close JSMR']].values

# Split Data into Training and Validation Sets and Standardize

In [ ]:
# 70% Train, 30% Val
total_ukuran_data = len(X)
ukuran_data_pelatihan = int(0.70 * total_ukuran_data)

X_Latih, Y_Latih = X[:ukuran_data_pelatihan], Y[:ukuran_data_pelatihan]
X_Validasi, Y_Validasi = X[ukuran_data_pelatihan:], Y[ukuran_data_pelatihan:]

skala_X = StandardScaler()
X_Latih_Standarisasi = skala_X.fit_transform(X_Latih)
X_Validasi_Standarisasi = skala_X.transform(X_Validasi)

skala_Y = StandardScaler()
Y_Latih_Standarisasi = skala_Y.fit_transform(Y_Latih)
Y_Validasi_Standarisasi = skala_Y.transform(Y_Validasi)

# Defining a Hyperparameter-Tuned Model Builder Function

In [ ]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_Latih.shape[1],)))

    for i in range(3):
        hp_units = hp.Int(f'units_{i}', min_value=64, max_value=512, step=32)

        hp_kernel_regularizer = hp.Float(f'kernel_regularizer_{i}', min_value=1e-6, max_value=1e-2, sampling='log')

        model.add(tf.keras.layers.Dense(units=hp_units, activation='relu',
                                        kernel_regularizer=regularizers.l2(hp_kernel_regularizer)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    model.add(tf.keras.layers.Dense(20, activation='relu'))

    hp_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mse', metrics=['mae'])

    return model

# Initializing the Hyperband Tuner for Model Optimization

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=1000,
                     factor=3,
                     directory='my_dir',
                     project_name='stock_price_prediction')

# Setting Up Early Stopping Callback

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)

# Running Hyperparameter Tuning with the Hyperband Tuner

In [ ]:
tuner.search(X_Latih_Standarisasi, Y_Latih_Standarisasi,
             validation_data=(X_Validasi_Standarisasi, Y_Validasi_Standarisasi),
             epochs=10000, callbacks=[stop_early])

# Retrieving and Displaying the Best Hyperparameters from Tuning Results

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"Optimal units: {best_hps.get('units_0')}, {best_hps.get('units_1')}, {best_hps.get('units_2')}")
print(f"Optimal dropout: {best_hps.get('dropout_0')}, {best_hps.get('dropout_1')}, {best_hps.get('dropout_2')}")
print(f"Optimal kernel_regularizer: {best_hps.get('kernel_regularizer_0')}, {best_hps.get('kernel_regularizer_1')}, {best_hps.get('kernel_regularizer_2')}")
print(f"Optimal learning rate: {best_hps.get('learning_rate')}")

# Building and Displaying the Summary of the Optimized Model

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
model.summary()

# Setting Up Model Checkpointing to Save the Best Model

In [ ]:
checkpoint_filepath = 'model_terbaik.keras'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min')

# Training the Model with Checkpointing and Validation Data

In [ ]:
history = model.fit(X_Latih_Standarisasi, Y_Latih_Standarisasi,
                    validation_data=(X_Validasi_Standarisasi, Y_Validasi_Standarisasi),
                    epochs=5000,
                    batch_size=32,
                    callbacks=[checkpoint])

# Plotting Training and Validation Loss Over Epochs

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs_range, loss, label='Training Loss', color='blue')
plt.plot(epochs_range, val_loss, label='Validation Loss', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

# Loading the Best Model from Checkpoint

In [ ]:
model_terbaik = tf.keras.models.load_model(checkpoint_filepath)

In [ ]:
model_terbaik.evaluate(X_Validasi_Standarisasi,Y_Validasi_Standarisasi)

# Making Predictions on New Data and Displaying Stock Prices

In [ ]:
data_baru = np.array([[15569.15, 6.25, 2.12]])
data_baru_standarisasi = skala_X.transform(data_baru)

prediksi_standarisasi = model_terbaik.predict(data_baru_standarisasi)
prediksi = skala_Y.inverse_transform(prediksi_standarisasi)

saham = ['Close CTRA', 'Close INDF', 'Close ASII', 'Close BSDE', 'Close ICBP',
          'Close KLBF', 'Close ITMG', 'Close JPFA', 'Close TLKM', 'Close ULTJ','Close ACES', 'Close TSPC',
          'Close SMAR', 'Close SMSM', 'Close JRPT', 'Close DUTI', 'Close EPMT', 'Close SMCB', 'Close PWON', 'Close JSMR']
hasil_prediksi = dict(zip(saham, prediksi[0]))

print("Prediksi Harga Saham Berdasarkan Data Baru:")
for stock, price in hasil_prediksi.items():
    print(f"{stock}: {price:.2f}")


# Saving Standard Scalers to Disk Using Joblib

In [ ]:
import joblib
joblib.dump(skala_X, 'skala_X.pkl')
joblib.dump(skala_Y, 'skala_Y.pkl')

In [ ]:
X_mean = skala_X.mean_
X_std_dev = skala_X.scale_
Y_mean = skala_Y.mean_
Y_std_dev = skala_Y.scale_
print("Mean dari X (fitur):", X_mean)
print("Standard Deviation dari X (fitur):", X_std_dev)
print("Mean dari Y (target):", Y_mean)
print("Standard Deviation dari Y (target):", Y_std_dev)

# Exporting the Best Model to a Specified Directory

In [ ]:
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print(f'export_path = {export_path}\n')

In [ ]:
model_terbaik.export(export_path)

# Listing Exported Model Files and Displaying Model Details

In [ ]:
!ls -l {export_path}

In [ ]:
!saved_model_cli show --dir {export_path} --all

# Zipping the Exported Model and Downloading It from Google Colab

In [ ]:
import shutil
from google.colab import files
shutil.make_archive("/tmp/model", 'zip', "/tmp/1")
files.download("/tmp/model.zip")

# Converting the Keras Model to TensorFlow Lite Format and Saving It

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_terbaik)
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
shutil.make_archive("/content/my_dir", 'zip', "/content/my_dir")
#files.download("/content/my_dir.zip")